In [26]:
import re
import pickle 
from itertools import chain
from datetime import datetime
from collections import defaultdict

from typing import List, Dict, Optional, Iterable, Tuple

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import tokenizers
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing

In [27]:
class Tokenizer:
    def __init__(self,
                 token_pattern: str = '\w+|[\!\?\,\.\-\:]',
                 eos_token: str = '<EOS>',
                 pad_token: str = '<PAD>',
                 unk_token: str = '<UNK>'):
        self.token_pattern = token_pattern
        self.eos_token = eos_token
        self.pad_token = pad_token
        self.unk_token = unk_token
        
        self.special_tokens = [self.eos_token, self.pad_token, self.unk_token]
        self.vocab = None
        self.inverse_vocab = None
    
    def text_preprocess(self, input_text: str) -> str:
        """ Предобрабатываем один текст """
        # input_text = ... # приведение к нижнему регистру
        input_text = input_text.lower()
        input_text = re.sub('\s+', ' ', input_text) # унифицируем пробелы
        input_text = input_text.strip()
        return input_text
    
    def build_vocab(self, corpus: List[str]) -> None:
        assert len(corpus)
        all_tokens = set()
        for text in corpus:
            all_tokens |= set(self._tokenize(text, append_eos_token=False))
        self.vocab = {elem: ind for ind, elem in enumerate(all_tokens)}
        special_tokens = [self.eos_token, self.unk_token, self.pad_token]
        for token in special_tokens:
            self.vocab[token] = len(self.vocab)
        self.inverse_vocab = {ind: elem for elem, ind in self.vocab.items()}
        return self
        
    def _tokenize(self, text: str, append_eos_token: bool = True) -> List[str]:
        text = self.text_preprocess(text)
        tokens = re.findall(self.token_pattern, text)
        if append_eos_token:
            tokens.append(self.eos_token)
        return tokens
    
    def encode(self, text: str, append_eos_token: bool = True) -> List[str]:
        """ Токенизируем текст """
        tokens = self._tokenize(text, append_eos_token)
        ids = [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]
        return ids
    
    def decode(self, input_ids: Iterable[int], remove_special_tokens: bool = False) -> str:
        assert len(input_ids)
        assert max(input_ids) < len(self.vocab) and min(input_ids) >= 0
        tokens = []
        for ind in input_ids:
            token = self.inverse_vocab[ind]
            if remove_special_tokens and token in self.special_tokens:
                continue
            tokens.append(token)
        text = ' '.join( tokens )
        return text
    
    def save(self, path: str) -> bool:
        data = {
            'token_pattern': self.token_pattern,
            'eos_token': self.eos_token,
            'pad_token': self.pad_token,
            'unk_token': self.unk_token,
            'special_tokens': self.special_tokens,
            'vocab': self.vocab,
            'inverse_vocab': self.inverse_vocab,
        }
        
        with open(path, 'wb') as fout:
            pickle.dump(data, fout)
            
        return True
        
    def load(self, path: str) -> bool:
        with open(path, 'rb') as fin:
            data = pickle.load(fin)
            
        self.token_pattern = data['token_pattern']
        self.eos_token = data['eos_token']
        self.pad_token = data['pad_token']
        self.unk_token = data['unk_token']
        self.special_tokens = data['special_tokens']
        self.vocab = data['vocab']
        self.inverse_vocab = data['inverse_vocab']

In [28]:
class GenerationConfig:
    def __init__(self, **kwargs):
        """
        Тут можно задать любые параметры и их значения по умолчанию
        Значения для стратегии декодирования decoding_strategy: ['max', 'top-p']
        """
        self.temperature = kwargs.pop("temperature", 1.0)
        self.max_tokens = kwargs.pop("max_tokens", 32)
        self.sample_top_p = kwargs.pop("sample_top_p", 0.9)
        self.decoding_strategy = kwargs.pop("decoding_strategy", 'max')
        self.remove_special_tokens = kwargs.pop("remove_special_tokens", False)
        self.validate()
        
    def validate(self):
        """ Здесь можно валидировать параметры """
        if not (1.0 > self.sample_top_p > 0):
            raise ValueError('sample_top_p')
        if self.decoding_strategy not in ['max', 'top-p']:
            raise ValueError('decoding_strategy')

In [30]:
def construct_model():
    config = {
        'temperature': 1.5,
        'max_tokens': 32,
        'sample_top_p': 0.9,
        'decoding_strategy': 'max',
    }

    stat_lm_path = 'models/stat_lm/stat_lm.pkl'
    tokenizer_path = 'models/stat_lm/tokenizer.pkl'
    
    tokenizer = Tokenizer()
    tokenizer.load(tokenizer_path)
        
    stat_lm = StatLM(tokenizer)
    stat_lm.load_stat(stat_lm_path)

    generation_config = GenerationConfig(temperature=config['temperature'],
                                         max_tokens=config['max_tokens'],
                                         sample_top_p=config['sample_top_p'],
                                         decoding_strategy=config['decoding_strategy'],
                                         remove_special_tokens=True)

    kwargs = {'generation_config': generation_config}
    return stat_lm, kwargs

### Обучаем на датасете шуток

In [31]:
generation_config = GenerationConfig(temperature = 1.5, max_tokens = 32, 
                                     sample_top_p = 0.9, decoding_strategy = 'top-p',
                                     remove_special_tokens=True)

# +- рандомно выставленные параметры

In [32]:
from datasets import load_dataset
dataset = load_dataset("IgorVolochay/russian_jokes") # датасет для обучения

dataset

In [33]:
text = dataset['train']['text'] 

In [34]:
tokenizer = Tokenizer().build_vocab(text)

In [35]:
dict(list(tokenizer.vocab.items())[:])

{'беседyют': 0,
 'макдональдса': 1,
 'двигатель': 2,
 'остановившись': 3,
 'выгребает': 4,
 'нер': 5,
 'членораздельную': 6,
 'вертикальный': 7,
 'вскрыты': 8,
 'содержащей': 9,
 'фунт': 10,
 'палермо': 11,
 'футбольной': 12,
 'oстоpожнее': 13,
 'пацанки': 14,
 'монархий': 15,
 'яндексбар': 16,
 'желаешь': 17,
 'прокладываешь': 18,
 'sp': 19,
 'посевной': 20,
 'транспортных': 21,
 'хaтки': 22,
 'стульчике': 23,
 'доками': 24,
 'твiй': 25,
 'вызовах': 26,
 'дорос': 27,
 'пролетариями': 28,
 'раскинувшийся': 29,
 'працюють': 30,
 'разговариваете': 31,
 'сиднея': 32,
 'гидpометеоpологических': 33,
 'ванн': 34,
 'полученный': 35,
 'конопляного': 36,
 'поднимусь': 37,
 'страха': 38,
 'многосерийную': 39,
 'короновирусокосный': 40,
 'тупенькие': 41,
 'басков': 42,
 'телезрителей': 43,
 'учёными': 44,
 'прибыльных': 45,
 'японо': 46,
 'катапультирование': 47,
 'любовн': 48,
 'проколи': 49,
 'вазу': 50,
 'разведзаданием': 51,
 'злой': 52,
 'пронзительный': 53,
 'рыло': 54,
 'pека': 55,
 'оптоп

In [40]:
# класс, который позволяем строить и использовать языковую модель на основе n-грамм
stat_lm = StatLM(tokenizer, context_size=12, alpha=0.25) 

stat_lm.train(text)

train lines:   0%|          | 0/150553 [00:00<?, ?it/s]

In [41]:
print(stat_lm.generate("вот такие пироги!", generation_config))

вот такие пироги ! невиновный совратила вытирает изученных лихую целом mark эйфорию предчувствии заканчивай миниатюрная видим хряпнули ракетчики михалковых посверлил инвалидах отважился родиля вывезти успейте шпротиков ошшш развлекает посткоитальной тaнк братоубийственная неудавшееся


In [38]:
# Сохраняем модель и токенезатор

tokenizer.save('models/stat_lm/tokenizer.pkl')
stat_lm.save_stat('models/stat_lm/stat_lm.pkl')

True

### смотрим как конструировать

In [39]:
model, kwargs = construct_model()

model.generate("дошик", **kwargs)

'дошик беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют беседyют'